In [128]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [129]:
file='../datasets/seeds.csv'
n_epochs=5
lr=0.05

In [130]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
#     print(dataset)
    return dataset

In [131]:
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    print("unique classes : ",unique)
    lookup = dict()
    #creat a dict for looking up all clasess
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [132]:
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

In [133]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [134]:
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))#get a random index from dataset
			fold.append(dataset_copy.pop(index))#remove that row from copy datset and append to fold
		dataset_split.append(fold)
	return dataset_split

In [135]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [136]:
def evaluate_algorithm(dataset, algorithm, n_folds, l_rate, n_epoch, n_hidden):
    folds = cross_validation_split(dataset, n_folds)
#     print(folds[0])
#     import pdb
#     pdb.set_trace()
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, l_rate, n_epoch, n_hidden)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [137]:
def back_propagation(train_set, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train_set[0]) - 1
	n_outputs = len(set([row[-1] for row in train_set]))
	network = initialize(n_inputs, n_hidden, n_outputs)
	train(network, train_set, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

# Initialize NN

In [138]:
def initialize(n_input,n_hidden,n_output):
    network=list()
    #for each neuron we have n_input+1 weights and we have n_hidden such "weights" list(array)
    h_layer=[{'weights':[random() for i in range(n_input+1)]} for i in range(n_hidden)]
    network.append(h_layer)
    #for each neuron we have n_hidden+1 weights and we have n_oputput such "weights" list(array)
    o_layer=[{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_output)]
    network.append(o_layer)
    return network

In [139]:
seed(1)
# Test that it works
# net=initialize(2,1,2)
# for l in net:
#     print(l)

# Acitvation

In [140]:
def activate(weights,inputs):
    activation=weights[-1] #last weight is bias weight
    for i in range(len(weights)-1):
        activation+=weights[i]*inputs[i]
    
    return activation

# Sigmoid Activation

In [141]:
def sigmoid(x): return 1.0/(1.0 + exp(-x))

# Forward Propagation

In [142]:
def forward(network,inputs):
    inputs=inputs
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activation=activate(neuron['weights'],inputs)
            neuron['output']=sigmoid(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    
    return inputs

# Test if it works

In [143]:
net=initialize(2,1,2)
row=[0,1,None]
out=forward(net,row)
print(out)

[0.6699713080575971, 0.7361939293022448]


# Backpropagation

## Sigmoid Derivative

In [144]:
def sigmoid_der(x): return x*(1.0 - x)

## Backpropagate Error

In [145]:
def backprop(network,expected):
    for i in reversed(range(len(network))): #traverse network in reverse direction
        layer=network[i] #a layer in nework 
        errors=list()
        if i != len(network) - 1:# if layer in not output layer
            for j in range(len(layer)): # iterate over all neurons
                error=0.0
                for neuron in network[i + 1]: #for each neuron of next layer
                    error += (neuron['weights'][j]*neuron['delta']) # error = weight_connecting_neuron_in_next_layer_to_current_neuron * 
                errors.append(error)
        #if oputput layer
        else:
            for j in range(len(layer)):#for each neuron in output layer
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])# error=y'-y
                
        # each neuron in current layer
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta'] = errors[j]*sigmoid_der(neuron['output'])# delta = error_of current_neuron* derivative_of_current_neuron_ouput



# Test It

In [146]:
net=initialize(2,1,2)
row=[1,0,None]
forward(net,row)
expected=[0,1]
backprop(net,expected)
for l in net:
    print(l)

[{'weights': [0.7887233511355132, 0.0938595867742349, 0.02834747652200631], 'output': 0.6936142046010635, 'delta': -0.011477619712406795}]
[{'weights': [0.8357651039198697, 0.43276706790505337], 'output': 0.7335023968859138, 'delta': -0.1433825771158816}, {'weights': [0.762280082457942, 0.0021060533511106927], 'output': 0.6296776889933221, 'delta': 0.08635312555373359}]


# Weight Update

In [147]:
def update(network,row,lr):
    for i in range(len(network)):
        inputs=row[:-1]
        if i!=0:
            inputs=[neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]+=lr*neuron['delta']*inputs[j]
            neuron['weights'][-1]+=lr*neuron['delta']
            

# Train

In [160]:
def train(network,train,lr,n_epochs,n_outputs):
    for epoch in range(n_epochs):
        sum_err=0.0
        for row in train:
            out=forward(network,row)
            expected=[0 for i in range(n_outputs)]
            expected[row[-1]]=1
            sum_err+=sum([(expected[i]-out[i])**2 for i in range(len(expected))])
            backprop(network,expected)
            update(network,row,lr)
        print('Epoch=%d, lr=%.3f, error=%.3f' %(epoch,lr,sum_err))

# Predict

In [161]:
def predict(network,row):
    out=forward(network,row)
    return out.index(max(out))

# Wheat Seed Dataset

In [162]:
dataset=load_csv(file);print(len(dataset));dataset[0:2]

211


[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8'],
 ['15.26', '14.84', '0.871', '5.763', '3.312', '2.221', '5.22', '1']]

In [163]:
dataset=dataset[1:]

In [164]:
for i in range(len(dataset[0])-1):# without considering last column
	str_column_to_float(dataset, i)#convert each row to float

In [165]:
dataset[0]

[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, '1']

In [166]:
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

unique classes :  {'1', '3', '2'}


{'1': 0, '3': 1, '2': 2}

In [167]:
dataset[0]

[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 0]

In [168]:
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [169]:
dataset[0]

[0.4409820585457979,
 0.5020661157024793,
 0.570780399274047,
 0.48648648648648646,
 0.48610121168923714,
 0.18930164220052273,
 0.3451501723289019,
 0]

In [170]:
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5

In [171]:
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Epoch=0, lr=0.300, error=142.829
Epoch=1, lr=0.300, error=114.676
Epoch=2, lr=0.300, error=112.208
Epoch=3, lr=0.300, error=107.932
Epoch=4, lr=0.300, error=100.179
Epoch=5, lr=0.300, error=88.952
Epoch=6, lr=0.300, error=77.954
Epoch=7, lr=0.300, error=69.818
Epoch=8, lr=0.300, error=63.915
Epoch=9, lr=0.300, error=59.036
Epoch=10, lr=0.300, error=54.514
Epoch=11, lr=0.300, error=50.159
Epoch=12, lr=0.300, error=46.020
Epoch=13, lr=0.300, error=42.213
Epoch=14, lr=0.300, error=38.829
Epoch=15, lr=0.300, error=35.904
Epoch=16, lr=0.300, error=33.424
Epoch=17, lr=0.300, error=31.346
Epoch=18, lr=0.300, error=29.610
Epoch=19, lr=0.300, error=28.159
Epoch=20, lr=0.300, error=26.939
Epoch=21, lr=0.300, error=25.907
Epoch=22, lr=0.300, error=25.026
Epoch=23, lr=0.300, error=24.266
Epoch=24, lr=0.300, error=23.605
Epoch=25, lr=0.300, error=23.025
Epoch=26, lr=0.300, error=22.511
Epoch=27, lr=0.300, error=22.052
Epoch=28, lr=0.300, error=21.638
Epoch=29, lr=0.300, error=21.264
Epoch=30, lr=0.

Epoch=256, lr=0.300, error=9.127
Epoch=257, lr=0.300, error=9.111
Epoch=258, lr=0.300, error=9.094
Epoch=259, lr=0.300, error=9.078
Epoch=260, lr=0.300, error=9.062
Epoch=261, lr=0.300, error=9.046
Epoch=262, lr=0.300, error=9.030
Epoch=263, lr=0.300, error=9.014
Epoch=264, lr=0.300, error=8.998
Epoch=265, lr=0.300, error=8.982
Epoch=266, lr=0.300, error=8.966
Epoch=267, lr=0.300, error=8.950
Epoch=268, lr=0.300, error=8.934
Epoch=269, lr=0.300, error=8.918
Epoch=270, lr=0.300, error=8.903
Epoch=271, lr=0.300, error=8.887
Epoch=272, lr=0.300, error=8.871
Epoch=273, lr=0.300, error=8.855
Epoch=274, lr=0.300, error=8.840
Epoch=275, lr=0.300, error=8.824
Epoch=276, lr=0.300, error=8.808
Epoch=277, lr=0.300, error=8.792
Epoch=278, lr=0.300, error=8.777
Epoch=279, lr=0.300, error=8.761
Epoch=280, lr=0.300, error=8.745
Epoch=281, lr=0.300, error=8.730
Epoch=282, lr=0.300, error=8.714
Epoch=283, lr=0.300, error=8.699
Epoch=284, lr=0.300, error=8.683
Epoch=285, lr=0.300, error=8.667
Epoch=286,

Epoch=14, lr=0.300, error=37.027
Epoch=15, lr=0.300, error=33.830
Epoch=16, lr=0.300, error=31.108
Epoch=17, lr=0.300, error=28.808
Epoch=18, lr=0.300, error=26.863
Epoch=19, lr=0.300, error=25.212
Epoch=20, lr=0.300, error=23.800
Epoch=21, lr=0.300, error=22.581
Epoch=22, lr=0.300, error=21.520
Epoch=23, lr=0.300, error=20.588
Epoch=24, lr=0.300, error=19.762
Epoch=25, lr=0.300, error=19.025
Epoch=26, lr=0.300, error=18.362
Epoch=27, lr=0.300, error=17.762
Epoch=28, lr=0.300, error=17.216
Epoch=29, lr=0.300, error=16.717
Epoch=30, lr=0.300, error=16.258
Epoch=31, lr=0.300, error=15.834
Epoch=32, lr=0.300, error=15.443
Epoch=33, lr=0.300, error=15.079
Epoch=34, lr=0.300, error=14.740
Epoch=35, lr=0.300, error=14.423
Epoch=36, lr=0.300, error=14.127
Epoch=37, lr=0.300, error=13.850
Epoch=38, lr=0.300, error=13.589
Epoch=39, lr=0.300, error=13.344
Epoch=40, lr=0.300, error=13.113
Epoch=41, lr=0.300, error=12.895
Epoch=42, lr=0.300, error=12.688
Epoch=43, lr=0.300, error=12.493
Epoch=44, 

Epoch=274, lr=0.300, error=5.084
Epoch=275, lr=0.300, error=5.071
Epoch=276, lr=0.300, error=5.059
Epoch=277, lr=0.300, error=5.046
Epoch=278, lr=0.300, error=5.034
Epoch=279, lr=0.300, error=5.021
Epoch=280, lr=0.300, error=5.008
Epoch=281, lr=0.300, error=4.996
Epoch=282, lr=0.300, error=4.983
Epoch=283, lr=0.300, error=4.971
Epoch=284, lr=0.300, error=4.958
Epoch=285, lr=0.300, error=4.946
Epoch=286, lr=0.300, error=4.933
Epoch=287, lr=0.300, error=4.921
Epoch=288, lr=0.300, error=4.908
Epoch=289, lr=0.300, error=4.896
Epoch=290, lr=0.300, error=4.883
Epoch=291, lr=0.300, error=4.871
Epoch=292, lr=0.300, error=4.859
Epoch=293, lr=0.300, error=4.846
Epoch=294, lr=0.300, error=4.834
Epoch=295, lr=0.300, error=4.822
Epoch=296, lr=0.300, error=4.809
Epoch=297, lr=0.300, error=4.797
Epoch=298, lr=0.300, error=4.785
Epoch=299, lr=0.300, error=4.773
Epoch=300, lr=0.300, error=4.761
Epoch=301, lr=0.300, error=4.748
Epoch=302, lr=0.300, error=4.736
Epoch=303, lr=0.300, error=4.724
Epoch=304,

Epoch=24, lr=0.300, error=29.944
Epoch=25, lr=0.300, error=29.427
Epoch=26, lr=0.300, error=28.939
Epoch=27, lr=0.300, error=28.473
Epoch=28, lr=0.300, error=28.025
Epoch=29, lr=0.300, error=27.594
Epoch=30, lr=0.300, error=27.176
Epoch=31, lr=0.300, error=26.771
Epoch=32, lr=0.300, error=26.375
Epoch=33, lr=0.300, error=25.990
Epoch=34, lr=0.300, error=25.613
Epoch=35, lr=0.300, error=25.245
Epoch=36, lr=0.300, error=24.886
Epoch=37, lr=0.300, error=24.534
Epoch=38, lr=0.300, error=24.190
Epoch=39, lr=0.300, error=23.855
Epoch=40, lr=0.300, error=23.527
Epoch=41, lr=0.300, error=23.208
Epoch=42, lr=0.300, error=22.898
Epoch=43, lr=0.300, error=22.596
Epoch=44, lr=0.300, error=22.303
Epoch=45, lr=0.300, error=22.019
Epoch=46, lr=0.300, error=21.744
Epoch=47, lr=0.300, error=21.478
Epoch=48, lr=0.300, error=21.221
Epoch=49, lr=0.300, error=20.973
Epoch=50, lr=0.300, error=20.734
Epoch=51, lr=0.300, error=20.503
Epoch=52, lr=0.300, error=20.282
Epoch=53, lr=0.300, error=20.068
Epoch=54, 

Epoch=281, lr=0.300, error=10.800
Epoch=282, lr=0.300, error=10.784
Epoch=283, lr=0.300, error=10.767
Epoch=284, lr=0.300, error=10.751
Epoch=285, lr=0.300, error=10.735
Epoch=286, lr=0.300, error=10.718
Epoch=287, lr=0.300, error=10.702
Epoch=288, lr=0.300, error=10.686
Epoch=289, lr=0.300, error=10.669
Epoch=290, lr=0.300, error=10.653
Epoch=291, lr=0.300, error=10.636
Epoch=292, lr=0.300, error=10.620
Epoch=293, lr=0.300, error=10.603
Epoch=294, lr=0.300, error=10.586
Epoch=295, lr=0.300, error=10.570
Epoch=296, lr=0.300, error=10.553
Epoch=297, lr=0.300, error=10.536
Epoch=298, lr=0.300, error=10.519
Epoch=299, lr=0.300, error=10.502
Epoch=300, lr=0.300, error=10.485
Epoch=301, lr=0.300, error=10.468
Epoch=302, lr=0.300, error=10.451
Epoch=303, lr=0.300, error=10.434
Epoch=304, lr=0.300, error=10.417
Epoch=305, lr=0.300, error=10.399
Epoch=306, lr=0.300, error=10.382
Epoch=307, lr=0.300, error=10.364
Epoch=308, lr=0.300, error=10.347
Epoch=309, lr=0.300, error=10.329
Epoch=310, lr=

Epoch=34, lr=0.300, error=19.971
Epoch=35, lr=0.300, error=19.705
Epoch=36, lr=0.300, error=19.460
Epoch=37, lr=0.300, error=19.233
Epoch=38, lr=0.300, error=19.021
Epoch=39, lr=0.300, error=18.824
Epoch=40, lr=0.300, error=18.640
Epoch=41, lr=0.300, error=18.466
Epoch=42, lr=0.300, error=18.303
Epoch=43, lr=0.300, error=18.149
Epoch=44, lr=0.300, error=18.003
Epoch=45, lr=0.300, error=17.865
Epoch=46, lr=0.300, error=17.733
Epoch=47, lr=0.300, error=17.608
Epoch=48, lr=0.300, error=17.489
Epoch=49, lr=0.300, error=17.375
Epoch=50, lr=0.300, error=17.266
Epoch=51, lr=0.300, error=17.161
Epoch=52, lr=0.300, error=17.061
Epoch=53, lr=0.300, error=16.964
Epoch=54, lr=0.300, error=16.871
Epoch=55, lr=0.300, error=16.781
Epoch=56, lr=0.300, error=16.695
Epoch=57, lr=0.300, error=16.611
Epoch=58, lr=0.300, error=16.530
Epoch=59, lr=0.300, error=16.452
Epoch=60, lr=0.300, error=16.376
Epoch=61, lr=0.300, error=16.302
Epoch=62, lr=0.300, error=16.231
Epoch=63, lr=0.300, error=16.161
Epoch=64, 

Epoch=301, lr=0.300, error=10.514
Epoch=302, lr=0.300, error=10.495
Epoch=303, lr=0.300, error=10.477
Epoch=304, lr=0.300, error=10.458
Epoch=305, lr=0.300, error=10.440
Epoch=306, lr=0.300, error=10.421
Epoch=307, lr=0.300, error=10.403
Epoch=308, lr=0.300, error=10.384
Epoch=309, lr=0.300, error=10.366
Epoch=310, lr=0.300, error=10.347
Epoch=311, lr=0.300, error=10.329
Epoch=312, lr=0.300, error=10.310
Epoch=313, lr=0.300, error=10.292
Epoch=314, lr=0.300, error=10.274
Epoch=315, lr=0.300, error=10.255
Epoch=316, lr=0.300, error=10.237
Epoch=317, lr=0.300, error=10.219
Epoch=318, lr=0.300, error=10.201
Epoch=319, lr=0.300, error=10.182
Epoch=320, lr=0.300, error=10.164
Epoch=321, lr=0.300, error=10.146
Epoch=322, lr=0.300, error=10.128
Epoch=323, lr=0.300, error=10.110
Epoch=324, lr=0.300, error=10.092
Epoch=325, lr=0.300, error=10.074
Epoch=326, lr=0.300, error=10.056
Epoch=327, lr=0.300, error=10.038
Epoch=328, lr=0.300, error=10.021
Epoch=329, lr=0.300, error=10.003
Epoch=330, lr=

Epoch=60, lr=0.300, error=19.569
Epoch=61, lr=0.300, error=19.471
Epoch=62, lr=0.300, error=19.375
Epoch=63, lr=0.300, error=19.282
Epoch=64, lr=0.300, error=19.190
Epoch=65, lr=0.300, error=19.101
Epoch=66, lr=0.300, error=19.014
Epoch=67, lr=0.300, error=18.929
Epoch=68, lr=0.300, error=18.845
Epoch=69, lr=0.300, error=18.764
Epoch=70, lr=0.300, error=18.684
Epoch=71, lr=0.300, error=18.605
Epoch=72, lr=0.300, error=18.528
Epoch=73, lr=0.300, error=18.453
Epoch=74, lr=0.300, error=18.379
Epoch=75, lr=0.300, error=18.307
Epoch=76, lr=0.300, error=18.236
Epoch=77, lr=0.300, error=18.166
Epoch=78, lr=0.300, error=18.097
Epoch=79, lr=0.300, error=18.030
Epoch=80, lr=0.300, error=17.964
Epoch=81, lr=0.300, error=17.898
Epoch=82, lr=0.300, error=17.834
Epoch=83, lr=0.300, error=17.771
Epoch=84, lr=0.300, error=17.709
Epoch=85, lr=0.300, error=17.648
Epoch=86, lr=0.300, error=17.588
Epoch=87, lr=0.300, error=17.529
Epoch=88, lr=0.300, error=17.471
Epoch=89, lr=0.300, error=17.413
Epoch=90, 

Epoch=329, lr=0.300, error=11.136
Epoch=330, lr=0.300, error=11.116
Epoch=331, lr=0.300, error=11.096
Epoch=332, lr=0.300, error=11.076
Epoch=333, lr=0.300, error=11.056
Epoch=334, lr=0.300, error=11.036
Epoch=335, lr=0.300, error=11.015
Epoch=336, lr=0.300, error=10.995
Epoch=337, lr=0.300, error=10.974
Epoch=338, lr=0.300, error=10.953
Epoch=339, lr=0.300, error=10.932
Epoch=340, lr=0.300, error=10.911
Epoch=341, lr=0.300, error=10.890
Epoch=342, lr=0.300, error=10.869
Epoch=343, lr=0.300, error=10.847
Epoch=344, lr=0.300, error=10.826
Epoch=345, lr=0.300, error=10.804
Epoch=346, lr=0.300, error=10.782
Epoch=347, lr=0.300, error=10.760
Epoch=348, lr=0.300, error=10.737
Epoch=349, lr=0.300, error=10.715
Epoch=350, lr=0.300, error=10.692
Epoch=351, lr=0.300, error=10.669
Epoch=352, lr=0.300, error=10.646
Epoch=353, lr=0.300, error=10.623
Epoch=354, lr=0.300, error=10.600
Epoch=355, lr=0.300, error=10.576
Epoch=356, lr=0.300, error=10.552
Epoch=357, lr=0.300, error=10.528
Epoch=358, lr=